<a href="https://colab.research.google.com/github/JamesBagley/James_Foundry_Tools/blob/master/Colaboratory%20M200%20Infinite%20DNA%20Processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Imports
import pandas as pd
import numpy as np

import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
from google.colab import files



In [ ]:
inputData = files.upload()
#@title Run Function
OutputFileName = 'JacksonvillePleaBargain.xlsx'  #@param {type: "string"}
HowCoolIsThis = 11  #@param {type: "slider", min: 0, max: 11}


data = pd.read_excel(next(iter(inputData))).loc[:,:'Sample ID']
"""
Created on Tue Aug 10 14:52:12 2021

@author: jbag2
"""

def read_nanoquant(data):
    ds = data.loc\
        [:,['Unnamed: 1','Conc ng/µl','Ratio','Sample ID']].rename\
            (columns={'Unnamed: 1':'NQ_plate_well'}).dropna\
                (subset=['NQ_plate_well']).reset_index(drop=True)
    cols = ds.apply(lambda x: int(x.NQ_plate_well[1:])+(x.name//16)*2, axis=1)
    rows = ds.apply(lambda x: x.NQ_plate_well[0], axis=1)
    original_mapping = \
        list(map(lambda x: x[0]+str(x[1]).zfill(2), zip(rows,cols)))
    ds = ds.assign(plate_assignment = original_mapping)
 
    ds = data.loc\
        [:,['Unnamed: 1','Conc ng/µl','Ratio','Sample ID']].rename\
            (columns={'Unnamed: 1':'NQ_plate_well'}).dropna\
                (subset=['NQ_plate_well']).reset_index(drop=True)
    cols = ds.apply(lambda x: int(x.NQ_plate_well[1:])+(x.name//16)*2, axis=1)
    rows = ds.apply(lambda x: x.NQ_plate_well[0], axis=1)
    original_mapping = \
        list(map(lambda x: x[0]+str(x[1]).zfill(2), zip(rows,cols)))
    ds = ds.assign(plate_assignment = original_mapping)
    ds = ds.set_index('plate_assignment')
    return(ds)

output = read_nanoquant(data).sort_values('plate_assignment')
output.to_excel(OutputFileName)
files.download(OutputFileName)

In [ ]:
#@title Generate Figures
output.plot(y='Conc ng/µl', kind='hist', xlim=[1,1500],
            xticks=[0,10,50,100,500,1000,1500], logx=True)
output.plot(y='Ratio', kind='hist', xlim=[1.5,2.1])

import plotly.express as px
platemap=pd.to_numeric(output['Conc ng/µl']).values.reshape([8,12])
fig = px.imshow(platemap, labels=dict(x="Column", y="Row",
                                      color="DNA Concentration (ng/µl)"),
                x='1,2,3,4,5,6,7,8,9,10,11,12'.split(','),
                y=list('ABCDEFGH'),
                width=1200, height=600)
fig.update_xaxes(side="top")
fig
